# Experiment 17: MISATA v3 - Privacy Fixed

## Issue Analysis
MIA AUC = 1.0 with noise meant the noise created EASILY DETECTABLE artifacts:
- Out-of-range values
- Unusual noise patterns
- Breaks natural data correlations

## v3 Solution: Smarter Privacy
1. **Resample instead of noise**: Add diversity via bootstrap, not Gaussian noise
2. **Manifold-preserving augmentation**: Stay within data support
3. **Mixup-style interpolation**: Combine samples for diversity
4. **K-anonymity inspired**: Ensure each synthetic record is near K real records

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)
print("Setup complete.")

## MISATA v3: Privacy-Preserving Synthesizer

In [ ]:
class MISATAv3Synthesizer:
    """
    MISATA v3 with PROPER privacy:
    
    Key insight: Don't add noise, add DIVERSITY through:
    1. Interpolation between samples (mixup)
    2. Staying within natural data bounds
    3. Controlled randomization of marginals
    """
    
    def __init__(self, target_col=None, task='classification',
                 privacy_level=0.3, random_state=42):
        """
        privacy_level: 0.0 = no privacy (exact marginals)
                       0.5 = balanced privacy-utility
                       1.0 = max privacy (more randomization)
        """
        self.target_col = target_col
        self.task = task
        self.privacy_level = privacy_level
        self.random_state = random_state
        
    def fit(self, df):
        self.columns = list(df.columns)
        self.n_train = len(df)
        self.train_data = df.values.copy()
        
        # Store column info
        self.col_info = {}
        for i, col in enumerate(self.columns):
            values = df[col].values
            self.col_info[col] = {
                'idx': i,
                'values': values.copy(),
                'sorted': np.sort(values),
                'min': values.min(),
                'max': values.max(),
                'std': values.std(),
                'unique': len(np.unique(values))
            }
        
        # Learn copula correlation
        uniform_df = df.copy()
        for col in self.columns:
            uniform_df[col] = stats.rankdata(df[col]) / (len(df) + 1)
        
        normal_df = uniform_df.apply(lambda x: stats.norm.ppf(np.clip(x, 0.001, 0.999)))
        corr_matrix = normal_df.corr().values
        corr_matrix = np.nan_to_num(corr_matrix, nan=0.0)
        np.fill_diagonal(corr_matrix, 1.0)
        
        # Positive definite adjustment
        eigvals, eigvecs = np.linalg.eigh(corr_matrix)
        eigvals = np.maximum(eigvals, 1e-6)
        corr_matrix = eigvecs @ np.diag(eigvals) @ eigvecs.T
        np.fill_diagonal(corr_matrix, 1.0)
        
        self.cholesky = np.linalg.cholesky(corr_matrix)
        
        # Target model
        if self.target_col and self.target_col in self.columns:
            feature_cols = [c for c in self.columns if c != self.target_col]
            if self.task == 'classification':
                self.target_model = GradientBoostingClassifier(
                    n_estimators=50, max_depth=4, random_state=self.random_state
                )
            else:
                from sklearn.ensemble import GradientBoostingRegressor
                self.target_model = GradientBoostingRegressor(
                    n_estimators=50, max_depth=4, random_state=self.random_state
                )
            self.target_model.fit(df[feature_cols], df[self.target_col])
            self.feature_cols = feature_cols
            self.target_rate = df[self.target_col].mean() if self.task == 'classification' else None
        
        return self
    
    def _interpolate_marginal(self, col_name, uniform_vals, rng):
        """
        Privacy-preserving marginal sampling via interpolation.
        Instead of exact quantile, interpolate between neighbors.
        """
        info = self.col_info[col_name]
        sorted_vals = info['sorted']
        n = len(sorted_vals)
        
        # Exact positions in sorted array
        positions = uniform_vals * (n - 1)
        lower_idx = np.floor(positions).astype(int)
        upper_idx = np.minimum(lower_idx + 1, n - 1)
        
        # Interpolation weights (with privacy randomization)
        base_weights = positions - lower_idx
        
        # Add controlled randomization based on privacy level
        if self.privacy_level > 0:
            # Randomly shift weights to create diversity
            noise = rng.uniform(-self.privacy_level, self.privacy_level, len(base_weights))
            weights = np.clip(base_weights + noise * 0.5, 0, 1)
        else:
            weights = base_weights
        
        # Linear interpolation between neighbors
        lower_vals = sorted_vals[lower_idx]
        upper_vals = sorted_vals[upper_idx]
        result = lower_vals * (1 - weights) + upper_vals * weights
        
        # Ensure in valid range
        result = np.clip(result, info['min'], info['max'])
        
        return result
    
    def sample(self, n_samples, seed=None):
        if seed is None:
            seed = self.random_state
        rng = np.random.default_rng(seed)
        
        # Correlated uniform sampling
        z = rng.standard_normal((n_samples, len(self.columns)))
        uniform = stats.norm.cdf(z @ self.cholesky.T)
        uniform = np.clip(uniform, 0.001, 0.999)
        
        # Privacy enhancement: add small perturbations to uniform values
        if self.privacy_level > 0:
            perturb = rng.uniform(-0.05 * self.privacy_level, 0.05 * self.privacy_level, uniform.shape)
            uniform = np.clip(uniform + perturb, 0.001, 0.999)
        
        synthetic_data = {}
        for i, col in enumerate(self.columns):
            if col == self.target_col:
                continue
            synthetic_data[col] = self._interpolate_marginal(col, uniform[:, i], rng)
        
        # Generate target
        if self.target_col and self.target_col in self.columns:
            X_synth = pd.DataFrame({c: synthetic_data[c] for c in self.feature_cols})
            if self.task == 'classification':
                probs = self.target_model.predict_proba(X_synth)[:, 1]
                # Smooth threshold with privacy
                if self.privacy_level > 0:
                    noise = rng.uniform(-0.05 * self.privacy_level, 0.05 * self.privacy_level, len(probs))
                    probs = np.clip(probs + noise, 0, 1)
                threshold = np.percentile(probs, (1 - self.target_rate) * 100)
                synthetic_data[self.target_col] = (probs >= threshold).astype(int)
            else:
                synthetic_data[self.target_col] = self.target_model.predict(X_synth)
        
        return pd.DataFrame(synthetic_data)[self.columns]

print("MISATA v3 Synthesizer defined.")

## Evaluation Functions

In [ ]:
def compute_metrics(real_train, real_test, synth):
    """Compute all metrics."""
    results = {}
    
    # 1. Correlation preservation (fixed)
    real_corr = real_train.corr().values
    synth_corr = synth.corr().values
    mask = ~np.eye(real_corr.shape[0], dtype=bool)
    
    real_flat = real_corr[mask]
    synth_flat = synth_corr[mask]
    valid = ~(np.isnan(real_flat) | np.isnan(synth_flat))
    
    results['pearson_corr'] = np.corrcoef(real_flat[valid], synth_flat[valid])[0, 1]
    tau, _ = stats.kendalltau(real_flat[valid], synth_flat[valid])
    results['kendall_tau'] = tau
    
    # 2. Marginal fidelity
    ks_scores = []
    for col in real_train.columns:
        stat, _ = stats.ks_2samp(real_train[col], synth[col])
        ks_scores.append(1 - stat)
    results['marginal_fidelity'] = np.mean(ks_scores)
    
    # 3. Privacy: MIA
    n_test = min(1000, len(real_train), len(synth))
    real_sample = real_train.sample(n_test, random_state=42)
    synth_sample = synth.sample(n_test, random_state=42)
    
    X_mia = pd.concat([real_sample, synth_sample], ignore_index=True)
    y_mia = np.array([1] * n_test + [0] * n_test)
    
    X_tr, X_te, y_tr, y_te = train_test_split(X_mia, y_mia, test_size=0.3, random_state=42)
    mia_model = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
    mia_model.fit(X_tr, y_tr)
    
    results['mia_auc'] = roc_auc_score(y_te, mia_model.predict_proba(X_te)[:, 1])
    results['mia_advantage'] = 2 * (results['mia_auc'] - 0.5)
    
    # 4. DCR
    scaler = StandardScaler()
    real_scaled = scaler.fit_transform(real_train)
    synth_scaled = scaler.transform(synth)
    
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(real_scaled)
    distances, _ = nn.kneighbors(synth_scaled)
    results['dcr_mean'] = np.mean(distances)
    results['dcr_5th'] = np.percentile(distances, 5)
    
    # 5. TSTR
    target = 'income'
    X_synth = synth.drop(target, axis=1)
    y_synth = synth[target]
    X_test = real_test.drop(target, axis=1)
    y_test = real_test[target]
    
    model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_synth, y_synth)
    results['tstr_auc'] = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    # TRTR
    model_real = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    model_real.fit(real_train.drop(target, axis=1), real_train[target])
    trtr = roc_auc_score(y_test, model_real.predict_proba(X_test)[:, 1])
    results['tstr_ratio'] = results['tstr_auc'] / trtr
    
    return results

print("Metrics defined.")

## Load Data and Test

In [ ]:
# Load Adult Census
print("Loading Adult Census...")
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status',
           'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
           'hours_per_week', 'native_country', 'income']
df_raw = pd.read_csv(url, names=columns, na_values=' ?', skipinitialspace=True)
df_raw = df_raw.dropna().reset_index(drop=True).sample(5000, random_state=SEED)
df_raw['income'] = (df_raw['income'] == '>50K').astype(int)

for col in ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']:
    df_raw[col] = LabelEncoder().fit_transform(df_raw[col].astype(str))

train_df, test_df = train_test_split(df_raw, test_size=0.2, random_state=SEED)
print(f"Train: {len(train_df)}, Test: {len(test_df)}")

In [ ]:
# Test different privacy levels
privacy_levels = [0.0, 0.1, 0.2, 0.3, 0.5, 0.7]
results = []

print("\nTesting privacy levels...\n")

for privacy in privacy_levels:
    print(f"Privacy level: {privacy}")
    
    synth = MISATAv3Synthesizer(target_col='income', privacy_level=privacy, random_state=SEED)
    synth.fit(train_df)
    df_synth = synth.sample(len(train_df))
    
    metrics = compute_metrics(train_df, test_df, df_synth)
    metrics['privacy_level'] = privacy
    results.append(metrics)
    
    print(f"  Kendall τ: {metrics['kendall_tau']:.4f}, MIA: {metrics['mia_auc']:.4f}, TSTR: {metrics['tstr_ratio']:.2%}")

results_df = pd.DataFrame(results)

In [ ]:
print("\n" + "="*80)
print("MISATA v3 RESULTS")
print("="*80)

display_cols = ['privacy_level', 'kendall_tau', 'mia_auc', 'mia_advantage', 'tstr_ratio', 'marginal_fidelity']
print(results_df[display_cols].to_string(index=False))

# Find optimal
viable = results_df[results_df['tstr_ratio'] > 0.90]
if len(viable) > 0:
    best = viable.loc[viable['mia_auc'].idxmin()]
    print(f"\n✓ Optimal Configuration:")
    print(f"  Privacy Level: {best['privacy_level']}")
    print(f"  MIA AUC: {best['mia_auc']:.4f} (lower = better)")
    print(f"  MIA Advantage: {best['mia_advantage']:.4f}")
    print(f"  TSTR Ratio: {best['tstr_ratio']:.2%}")
    print(f"  Kendall τ: {best['kendall_tau']:.4f}")

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Privacy-Utility Tradeoff
ax1 = axes[0]
ax1.plot(results_df['privacy_level'], results_df['mia_auc'], 'b-o', label='MIA AUC (↓better)', linewidth=2, markersize=8)
ax1.plot(results_df['privacy_level'], results_df['tstr_ratio'], 'g-s', label='TSTR Ratio', linewidth=2, markersize=8)
ax1.axhline(y=0.5, color='blue', linestyle='--', alpha=0.5)
ax1.axhline(y=0.9, color='green', linestyle='--', alpha=0.5)
ax1.set_xlabel('Privacy Level', fontsize=12)
ax1.set_ylabel('Score', fontsize=12)
ax1.set_title('Privacy-Utility Tradeoff', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0.4, 1.05)

# Plot 2: Correlation Preservation
ax2 = axes[1]
ax2.plot(results_df['privacy_level'], results_df['pearson_corr'], 'r-o', label='Pearson', linewidth=2, markersize=8)
ax2.plot(results_df['privacy_level'], results_df['kendall_tau'], 'b-s', label='Kendall τ', linewidth=2, markersize=8)
ax2.set_xlabel('Privacy Level', fontsize=12)
ax2.set_ylabel('Correlation Similarity', fontsize=12)
ax2.set_title('Correlation Preservation', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# Plot 3: Distance to Real Data
ax3 = axes[2]
ax3.plot(results_df['privacy_level'], results_df['dcr_5th'], 'purple', marker='o', linewidth=2, markersize=8)
ax3.set_xlabel('Privacy Level', fontsize=12)
ax3.set_ylabel('DCR 5th Percentile', fontsize=12)
ax3.set_title('Distance to Closest Real Record', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('misata_v3_results.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n✓ Saved misata_v3_results.png")

In [ ]:
# Save results
results_df.to_csv('misata_v3_privacy_results.csv', index=False)

print("\n" + "="*80)
print("EXPERIMENT 17 COMPLETE")
print("="*80)
print("\nKey Improvements in v3:")
print("  1. Interpolation-based marginal sampling (no out-of-range artifacts)")
print("  2. Subtle uniform perturbations (preserves correlation structure)")
print("  3. Controlled privacy-utility tradeoff")
print("\nFiles saved:")
print("  - misata_v3_results.png")
print("  - misata_v3_privacy_results.csv")